In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
import h5py
import os
import itertools
from utils import *

In [19]:
f1 = h5py.File('/Users/theoares/Dropbox (MIT)/research/miscellaneous/meas/dimitra_stout.h5', 'r')
x1 = f1['prop/p1111'][()]
print(x1)

[[[[ 2.67635257e+06-2.71955464e-07j  3.09306728e+01-5.08168131e+01j
    -3.07547630e+01-1.18227427e+02j]
   [ 3.09306573e+01+5.08167823e+01j  2.67659741e+06-9.23718351e-07j
     7.50992824e+01+7.56172887e+01j]
   [-3.07547463e+01+1.18227409e+02j  7.50992742e+01-7.56172561e+01j
     2.67610836e+06-1.30843319e-07j]]

  [[ 4.51310933e+01-7.95631893e+01j -1.88264667e+02+1.15218332e+02j
    -1.00151776e+02-7.82384620e+01j]
   [-4.05096635e+01-1.58242908e+01j  4.98104095e+01+1.94267901e+01j
    -9.18121290e+01+6.30482264e+00j]
   [-2.29875584e+01+6.09921820e+01j  1.61262363e+02-8.61655576e+01j
     4.08579516e+01+5.15867380e+01j]]

  [[ 4.54742399e+02+2.92657372e+02j -1.44460315e+02+7.88474807e+01j
     1.19716319e+02-7.88732376e+00j]
   [-6.59101867e+00-1.79422641e+02j  2.74390533e+02+1.00014956e+02j
     1.62382407e+02-3.47025242e+02j]
   [-6.15561853e+01+3.41243997e+02j -1.66877933e+02+1.37602558e+02j
    -2.79661951e+02+2.04805764e+02j]]

  [[-5.26722212e+01+1.68639334e+02j  2.33346658e+

In [20]:
print('Trace of propagator color indices')
print(np.einsum('ijaa->ij', x1))

Trace of propagator color indices
[[ 8.02905834e+06-1.32651713e-06j  1.35799454e+02-8.54966116e+00j
   4.49470981e+02+5.97478093e+02j  6.21425743e+02+2.37566881e+02j]
 [ 1.35799501e+02+8.54974944e+00j  8.02921479e+06+9.31817876e-08j
   7.21405047e+02-1.21758205e+02j -4.27269816e+02+6.14059043e+02j]
 [-4.49470930e+02+5.97478103e+02j -7.21405012e+02-1.21758300e+02j
   8.02916216e+06-1.44529484e-06j  9.79258654e+01+2.03029072e+02j]
 [-6.21425716e+02+2.37566737e+02j  4.27269798e+02+6.14059188e+02j
   9.79259785e+01-2.03028931e+02j  8.02939436e+06-2.00702764e-06j]]


In [17]:
f2 = h5py.File('/Users/theoares/Dropbox (MIT)/research/miscellaneous/meas/my_stout.h5', 'r')
x2 = f2['prop/p1111'][()]
print(x2)

[[[[ 2.76418348e+06+1.95399252e-13j  1.04567816e+02-8.14126971e+01j
    -9.53409493e+01-1.60266600e+02j]
   [ 1.04567816e+02+8.14126971e+01j  2.76436953e+06-1.33937306e-12j
     7.72438998e+01+8.60521162e+01j]
   [-9.53409493e+01+1.60266600e+02j  7.72438998e+01-8.60521162e+01j
     2.76384312e+06+8.86402063e-13j]]

  [[-3.40058018e+01-1.18625906e+02j -1.71702780e+02-1.64123448e+01j
    -1.15542677e+02-2.32837918e+01j]
   [ 2.31320765e+01-3.69532062e+01j  4.56592857e+01-2.91784716e+01j
    -1.36046787e+02+1.78486695e+01j]
   [-3.17397139e+00+4.88086427e+01j  9.71403633e+01-6.90945982e+01j
    -2.45752995e+01-2.86403300e+00j]]

  [[ 3.10570600e+02+2.63379064e+02j -5.93973832e+01-3.82688346e+01j
     1.10023442e+02+9.22391596e+01j]
   [-1.47978836e+01-1.47659202e+02j  1.97968159e+02+8.74040301e+01j
     1.09979504e+02-3.08429795e+02j]
   [-6.25233851e+01+3.06515635e+02j -1.86663823e+02+8.59968918e+01j
    -1.38486139e+02+1.83405996e+02j]]

  [[-1.43473279e+02+1.34190277e+02j  2.00140734e+

In [20]:
print('Trace of propagator color indices')
print(np.einsum('ijaa->ij', x1))

Trace of propagator color indices
[[ 8.02905834e+06-1.32651713e-06j  1.35799454e+02-8.54966116e+00j
   4.49470981e+02+5.97478093e+02j  6.21425743e+02+2.37566881e+02j]
 [ 1.35799501e+02+8.54974944e+00j  8.02921479e+06+9.31817876e-08j
   7.21405047e+02-1.21758205e+02j -4.27269816e+02+6.14059043e+02j]
 [-4.49470930e+02+5.97478103e+02j -7.21405012e+02-1.21758300e+02j
   8.02916216e+06-1.44529484e-06j  9.79258654e+01+2.03029072e+02j]
 [-6.21425716e+02+2.37566737e+02j  4.27269798e+02+6.14059188e+02j
   9.79259785e+01-2.03028931e+02j  8.02939436e+06-2.00702764e-06j]]


In [18]:
print('Trace of propagator color indices')
print(np.einsum('ijaa->ij', x2))

Trace of propagator color indices
[[ 8.29239613e+06-2.57571742e-13j -1.29218157e+01-1.50668411e+02j
   3.70052619e+02+5.34189089e+02j  4.37900720e+02+1.20255220e+02j]
 [-1.29218157e+01+1.50668411e+02j  8.29256941e+06+3.81472631e-12j
   4.11426405e+02-1.20461117e+02j -4.19680275e+02+5.38167310e+02j]
 [-3.70052619e+02+5.34189089e+02j -4.11426405e+02-1.20461117e+02j
   8.29258614e+06-1.09068310e-12j  1.05288100e+00-1.09142459e+01j]
 [-4.37900720e+02+1.20255220e+02j  4.19680275e+02+5.38167310e+02j
   1.05288100e+00+1.09142459e+01j  8.29250083e+06-3.54472007e-12j]]


In [15]:
print('Trace of propagator color indices')
print(np.einsum('ijaa->ij', x3))

Trace of propagator color indices
[[ 8.02905834e+06-1.33869253e-06j  1.35799454e+02-8.54966127e+00j
   4.49470981e+02+5.97478093e+02j  6.21425743e+02+2.37566881e+02j]
 [ 1.35799501e+02+8.54974950e+00j  8.02921479e+06+1.45695154e-08j
   7.21405047e+02-1.21758205e+02j -4.27269816e+02+6.14059043e+02j]
 [-4.49470930e+02+5.97478103e+02j -7.21405012e+02-1.21758300e+02j
   8.02916216e+06-1.54334050e-06j  9.79258654e+01+2.03029072e+02j]
 [-6.21425716e+02+2.37566737e+02j  4.27269797e+02+6.14059188e+02j
   9.79259785e+01-2.03028931e+02j  8.02939436e+06-2.00845035e-06j]]


In [14]:
f3 = h5py.File('/Users/theoares/Dropbox (MIT)/research/miscellaneous/meas/qlua_stout.h5', 'r')
x3 = f3['prop/p1111'][()]
print(x3)

[[[[ 2.67635257e+06-2.87602258e-07j  3.09306728e+01-5.08168131e+01j
    -3.07547630e+01-1.18227427e+02j]
   [ 3.09306572e+01+5.08167823e+01j  2.67659741e+06-8.99551655e-07j
     7.50992825e+01+7.56172886e+01j]
   [-3.07547463e+01+1.18227409e+02j  7.50992742e+01-7.56172561e+01j
     2.67610836e+06-1.51538621e-07j]]

  [[ 4.51310932e+01-7.95631893e+01j -1.88264667e+02+1.15218332e+02j
    -1.00151776e+02-7.82384620e+01j]
   [-4.05096635e+01-1.58242908e+01j  4.98104094e+01+1.94267901e+01j
    -9.18121290e+01+6.30482264e+00j]
   [-2.29875584e+01+6.09921820e+01j  1.61262363e+02-8.61655576e+01j
     4.08579517e+01+5.15867380e+01j]]

  [[ 4.54742399e+02+2.92657372e+02j -1.44460315e+02+7.88474807e+01j
     1.19716319e+02-7.88732380e+00j]
   [-6.59101871e+00-1.79422641e+02j  2.74390533e+02+1.00014956e+02j
     1.62382407e+02-3.47025242e+02j]
   [-6.15561853e+01+3.41243997e+02j -1.66877933e+02+1.37602558e+02j
    -2.79661951e+02+2.04805764e+02j]]

  [[-5.26722211e+01+1.68639333e+02j  2.33346658e+

In [21]:
f4 = h5py.File('/Users/theoares/Dropbox (MIT)/research/miscellaneous/meas/qlua_saved_stout.h5', 'r')
x4 = f4['prop/p1111'][()]
print(x4)

[[[[ 2.67635257e+06-2.74238349e-07j  3.09306728e+01-5.08168130e+01j
    -3.07547630e+01-1.18227427e+02j]
   [ 3.09306572e+01+5.08167823e+01j  2.67659741e+06-9.12586311e-07j
     7.50992824e+01+7.56172887e+01j]
   [-3.07547463e+01+1.18227409e+02j  7.50992742e+01-7.56172560e+01j
     2.67610836e+06-1.66294550e-07j]]

  [[ 4.51310932e+01-7.95631894e+01j -1.88264667e+02+1.15218332e+02j
    -1.00151776e+02-7.82384620e+01j]
   [-4.05096635e+01-1.58242908e+01j  4.98104095e+01+1.94267901e+01j
    -9.18121289e+01+6.30482262e+00j]
   [-2.29875584e+01+6.09921820e+01j  1.61262363e+02-8.61655576e+01j
     4.08579517e+01+5.15867380e+01j]]

  [[ 4.54742399e+02+2.92657372e+02j -1.44460315e+02+7.88474807e+01j
     1.19716319e+02-7.88732375e+00j]
   [-6.59101876e+00-1.79422641e+02j  2.74390533e+02+1.00014956e+02j
     1.62382407e+02-3.47025242e+02j]
   [-6.15561853e+01+3.41243997e+02j -1.66877933e+02+1.37602558e+02j
    -2.79661951e+02+2.04805764e+02j]]

  [[-5.26722212e+01+1.68639333e+02j  2.33346657e+

In [22]:
print('Trace of propagator color indices')
print(np.einsum('ijaa->ij', x4))

Trace of propagator color indices
[[ 8.02905834e+06-1.35311921e-06j  1.35799454e+02-8.54966130e+00j
   4.49470981e+02+5.97478093e+02j  6.21425743e+02+2.37566881e+02j]
 [ 1.35799501e+02+8.54974948e+00j  8.02921479e+06+1.75051973e-08j
   7.21405047e+02-1.21758205e+02j -4.27269816e+02+6.14059043e+02j]
 [-4.49470930e+02+5.97478103e+02j -7.21405012e+02-1.21758300e+02j
   8.02916216e+06-1.50735501e-06j  9.79258654e+01+2.03029072e+02j]
 [-6.21425716e+02+2.37566737e+02j  4.27269797e+02+6.14059188e+02j
   9.79259785e+01-2.03028931e+02j  8.02939436e+06-2.02147789e-06j]]


## Playing with data

In [2]:
def read_cfg_Npt_explicit(folder, stem, N, n_t, cfg):
#     file = folder + '/' + stem + '.1460'
    file = folder + '/' + stem + '.' + str(cfg)
    if N == 2:
        Cnpt = np.zeros((n_t, n_t), dtype = np.complex64)
    else:
        Cnpt = np.zeros((24, n_t, n_t, n_t), dtype = np.complex64)
    with open(file) as f:
        lines = [line.rstrip() for line in f]
    for l in lines:
        x = l.split()
        if N == 2:
            tsrc, tsnk = int(x[0]), int(x[1])
            entry = np.complex(float(x[2]), float(x[3]))
            Cnpt[idx, tsrc, tsnk] = entry
        else:           # read 3-pt contraction structure
            tminus, tx, tplus = int(x[0]), int(x[1]), int(x[2])
            for i in range(24):
                re, im = float(x[2 * i + 3]), float(x[2 * i + 4])
                Cnpt[i, tminus, tx, tplus] = np.complex(re, im)
    return Cnpt

In [3]:
config = '24I/ml_0p01'
resultsA = '/Users/theoares/Pi_Pi/' + config + '/resultsA'
stem_3pt = 'pion_0vbb_4quark'
l, t = 24, 64
L = Lattice(l, t)

In [4]:
C3pt = read_cfg_Npt_explicit(resultsA, stem_3pt, 3, L.T, 1580)

In [ ]:
# x = np.zeros((L.T), dtype = np.complex64)
# for t in range(L.T):
#     x[t] = C3pt[0, t, 7, t]
# print(np.average(x))

In [ ]:
# x = []
# for t1 in range(L.T):
#     for t2 in range(t1, L.T):
#         if np.abs(t1 - t2) == 1:
#             x.append(C3pt[0, t1, 1, t2])    # should avg to first operator on line 1, 1

In [ ]:
# sum(x) / len(x)

In [ ]:
# C3pt_tavg = np.zeros((24, L.T, L.T), dtype = np.complex64)    # (tx, |t+ - t-|)
# # cnt = np.zeros((L.T, L.T))
# for tminus, tx, tplus in itertools.product(range(L.T), repeat = 3):
#     t1 = (tx - tminus) % L.T
#     t2 = (tplus - tminus) % L.T
#     if t1 == 1 and t2 == 2:
#         print((tminus, tx, tplus))
#     C3pt_tavg[:, t1, t2] += C3pt[:, tminus, tx, tplus]    # need to average about tminus = 0 first
# C3pt_tavg = C3pt_tavg / L.T

In [ ]:
# C3pt.shape[:-1]

In [5]:
C3pt_tavg = np.zeros((24, L.T, L.T), dtype = np.complex64)    # (tx, |t+ - t-|)
cnt = np.zeros((L.T, L.T))
for tminus, tx, tplus in itertools.product(range(L.T), repeat = 3):
    C3pt_tavg[:, tminus, tplus] += C3pt[:, tminus, tx, tplus]    # need to average about tminus = 0 first
    cnt[tminus, tplus] += 1
for tminus, tplus in itertools.product(range(L.T), repeat = 2):
    C3pt_tavg[:, tminus, tplus] = C3pt_tavg[:, tminus, tplus] / cnt[tminus, tplus]

In [6]:
C3pt_refl = np.zeros((24, L.T, L.T), dtype = np.complex64)
for t1, t2 in itertools.product(range(L.T), repeat = 2):
    C3pt_refl[:, t1, t2] = C3pt_tavg[:, (L.T - t1) % L.T, (L.T - t2) % L.T]

In [7]:
# first t index is in the file name, second is the label in the file
C_sep = np.zeros((24, L.T, L.T), dtype = np.complex64)
C_sep_refl = np.zeros((24, L.T, L.T), dtype = np.complex64)
for sep, t in itertools.product(range(L.T), repeat = 2):
    C_sep[:, sep, t] = C3pt_tavg[:, (L.T - sep) % L.T, (L.T - sep + t) % L.T]
    C_sep_refl[:, sep, t] = C3pt_refl[:, (L.T - sep) % L.T, (L.T - sep + t) % L.T]

In [ ]:
C_sep_refl[:, 35, 6]

Ex: Line 2 42 in pion_0vbb_4quark.1460 is 2.576e2 0 3.21e2 ~0, line 2 in pion_0vbb_4quark.42.1460 is 2.27e4 0 2.97e4 ~0, so these seem to be different things. I'd assume the average is done in a different way or something, but would like to figure out how to get each of these numbers from the raw data

Maybe try to fold over the x insertion and / or the midpoint of the lattice and see what we get?

_ref is the reflected file, see resultsA.tavg/run.sh.

The time averager is now giving me what David's files have, it seems to be averaged over tx.
Next: do reflections, and figure out the difference between pion_0vbb_4quark.1460 and pion_0vbb_4quark.1.1460 (and all other separations) in resultsA.tavg

So I found a hidden .awk file in one of the directories in the project that does the time averaging, and managed to use that to replicate the data in the time-averaged folder in my python script. It looks like the time averaging is actually over operator insertion times-- unless David mislabeled the time labels in his writeup for the 3 point function with (t-, tx, t+), the time averaged file at (t1, t2) is actually $(t_-, t_+)$ averaged over tx. I’m not sure why he says the labels in the time averaged files are $(t_x, |t_+ - t_-|)$, does this make any sense to you?

First notes: 
pion_0vbb_4quark.1460: Label 63, 63 matches up with pion_0vbb_4quark.1.1460 Label 0, except the imaginary parts are sometimes opposite. Same with the following:

pion_0vbb_4quark.SEP.CFG: Rows labeled by $\Delta t$ and the file is labeled by $t_{sep}$, where we're looking at the information for $t_- = T - t_{sep}$ and $t_+ = t_- + \Delta t$. 

Label in .1460 <--> Label in .1.1460
63, 63         <-->         0
63, 0          <-->         1
1, 2 and 63, 1 <-->         2

Label in .1460 <--> Label in .9.1460
55, 57         <-->         2

Label in .1460 <--> Label in .24.1460
40, 47         <-->         7

TODO: I don't need C_sep, it's not right. To get pion_0vbb_4quark.SEP.CFG at line t, just index C3pt_tavg[CFG, :, SEP, t] (where CFG just indexes the config number in the files)

In [ ]:
# now figure out the .12.1460 files, etc.

In [11]:
C3pt_tavg[:, 1, 8]

array([ 3.13151031e+05+0.0000000e+00j,  3.91712344e+05-9.7637776e-15j,
        1.83089188e+05-7.6384436e-16j,  2.63706625e+05+2.6705932e-16j,
        7.27447250e+05+0.0000000e+00j,  4.41469156e+05+7.5880532e-15j,
        3.13100781e+05+8.3280766e-16j,  2.83541344e+05-4.3183105e-16j,
       -8.93180566e+03-4.7593675e-16j, -1.23001734e+05+5.2565111e-15j,
       -5.67667695e+04-2.7303000e-15j, -4.25199000e+05+1.7879576e-15j,
        2.18995789e-16+1.5303595e+02j,  7.57056427e+01+3.6468773e+01j,
       -4.15108189e-15+1.8275929e+02j,  3.71212067e+02-4.9737854e+01j,
       -4.55564516e-15-5.3560299e+01j, -7.57056427e+01+3.6468773e+01j,
        1.70589820e-15-2.5569691e+02j, -3.71212067e+02-4.9737854e+01j,
        3.07376465e+04+1.8489937e-15j,  1.37021453e+05+3.4889795e-16j,
        4.27470742e+04-3.7518219e-17j,  4.03393250e+05+1.3814176e-16j],
      dtype=complex64)

In [12]:
# C3pt_tavg[:, 18, 7]

In [13]:
C_sep[:, 1, 8]

array([ 3.13151031e+05+0.0000000e+00j,  3.91712344e+05+9.7637776e-15j,
        1.83089219e+05+7.6384436e-16j,  2.63706625e+05-2.6705932e-16j,
        7.27447250e+05+0.0000000e+00j,  4.41469156e+05-7.5880532e-15j,
        3.13100781e+05-8.3280766e-16j,  2.83541375e+05+4.3183105e-16j,
       -8.93180469e+03-4.7593670e-16j, -1.23001734e+05-1.6767737e-16j,
       -5.67667656e+04-2.2600357e-15j, -4.25199031e+05-1.8889898e-15j,
       -4.55564516e-15-5.3560276e+01j, -7.57055817e+01+3.6468781e+01j,
       -1.36809861e-15-2.5569682e+02j, -3.71211853e+02-4.9737846e+01j,
        2.18995922e-16+1.5303595e+02j,  7.57055817e+01+3.6468781e+01j,
       -4.96224384e-15+1.8275925e+02j,  3.71211853e+02-4.9737846e+01j,
        3.07376465e+04+1.8489944e-15j,  1.37021453e+05-9.2718185e-16j,
        4.27470742e+04+4.5166973e-16j,  4.03393188e+05-1.3064698e-15j],
      dtype=complex64)